In [ ]:
#will use VGG16 conv base and will extract the feature and then flatten it and branching and then put in FC


In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

 96% 318M/331M [00:01<00:00, 241MB/s]
100% 331M/331M [00:01<00:00, 216MB/s]


In [3]:
import zipfile
zip=zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
len(age)

23708

In [8]:
df= pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [9]:
df.shape

(23708, 3)

In [10]:
df.head()

,age,gender,img
0,36,1,36_1_0_20170116165722892.jpg.chip.jpg
1,36,0,36_0_4_20170113133645566.jpg.chip.jpg
2,34,0,34_0_3_20170119200758796.jpg.chip.jpg
3,53,1,53_1_0_20170104184217989.jpg.chip.jpg
4,1,0,1_0_3_20161219225633088.jpg.chip.jpg


In [11]:
#train test split
train_df=df.sample(frac=1,random_state=0).iloc[:20000]
test_df= df.sample(frac=1,random_state=0).iloc[20000:]

In [12]:
train_df.shape

(20000, 3)

In [13]:
#applying data augmentation
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [14]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_dataframe(
        train_df,
        directory='/content/utkface_aligned_cropped/UTKFace',
        x_col='img',
        y_col=['age','gender'],
        target_size=(200, 200),
        #batch_size=32,
        class_mode='multi_output')
test_generator = test_datagen.flow_from_dataframe(
        test_df,
        directory='/content/utkface_aligned_cropped/UTKFace',
        x_col='img',
        y_col=['age','gender'],
        target_size=(200, 200),
        #batch_size=32,
        class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [17]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [18]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 0s 0us/step


In [20]:
resnet.trainable=False

output=resnet.layers[-1].output
flatten=Flatten()(output) #flatten layers is getting output of resnet excluding last layers as input

dense1=Dense(512,activation='relu')(flatten)
dense2=Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [21]:
model=Model(inputs=resnet.input,outputs=[output1,output2])

In [22]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [24]:
model.fit(train_generator,batch_size=32,epochs=10,validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 244s 375ms/step - loss: 97.2063 - age_loss: 15.5202 - gender_loss: 0.8251 - age_mae: 15.5202 - gender_accuracy: 0.5096 - val_loss: 83.2302 - val_age_loss: 14.7368 - val_gender_loss: 0.6919 - val_age_mae: 14.7368 - val_gender_accuracy: 0.5348
Epoch 2/10
625/625 [==============================] - 229s 367ms/step - loss: 83.5263 - age_loss: 14.9793 - gender_loss: 0.6924 - age_mae: 14.9793 - gender_accuracy: 0.5204 - val_loss: 83.0798 - val_age_loss: 14.6658 - val_gender_loss: 0.6911 - val_age_mae: 14.6658 - val_gender_accuracy: 0.5345
Epoch 3/10
625/625 [==============================] - 228s 364ms/step - loss: 84.1527 - age_loss: 14.9356 - gender_loss: 0.6992 - age_mae: 14.9356 - gender_accuracy: 0.5195 - val_loss: 82.9033 - val_age_loss: 14.4553 - val_gender_loss: 0.6914 - val_age_mae: 14.4553 - val_gender_accuracy: 0.5345
Epoch 4/10
625/625 [==============================] - 228s 366ms/step - loss: 83.2931 - age_loss: 14.7365 - gend